In [1]:
## Importing Necessary Modules 

from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
## getting the appropriate website parsed into hmtl
### Defining a function which takes in a url and outputs soup
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return(soup)


In [89]:
#format url based off the race 
def format_url(race = 'tour-de-france', year = 2023, stage = None):
    
    url_basic = 'https://www.procyclingstats.com/race/{}/{}'
    
    if stage:
        return url_basic.format(race, year) + '/stage-{}'.format(stage)
    
    return url_basic.format(race, year)


In [120]:
def extract_result(url):
    
    #soupify
    soup = get_soup(url)
    
    headers = []
    data = []
    
    #find rows
    rows = soup.find_all('tr')
    
    #generate header
    header_row = rows[0]
    for header in header_row.find_all('th'):
        headers.append(header.text) 
        data.append([])
    
    #generate rows
    for data_row in rows[1:11]:
        row = data_row.find_all('td')
        for i in range(len(row)):
            if len(row) != len(data):
                continue
            else:
                data[i].append(row[i].text.strip())
    
    #put into dataframe
    data_dict = {}
    for i in range(len(headers)):
        data_dict[headers[i]] = data[i]
    df = pd.DataFrame(data_dict)
    return(df)


In [121]:
all_stages_data = {}
stage_number = 1
url_basic = 'https://www.procyclingstats.com/race/tour-de-france/2023/stage-'
for stage_number in range(1,22):
    url = url_basic + str(stage_number)
    all_stages_data[str(stage_number)] = extract_result(url)

    
all_stages_data['19'].head(10)

,Rnk,GC,Timelag,BIB,H2H,Specialty,Rider,Age,Team,UCI,Pnt,,Time
0,1,67,+3:19:36,67,,Classic,MOHORIČ Matej Bahrain - Victorious,28,Bahrain - Victorious,210,100,10″,3:31:0210″
1,2,80,+3:45:12,52,,TT,ASGREEN Kasper Soudal Quick-Step,28,Soudal Quick-Step,150,70,6″,",,0:006″"
2,3,17,+59:46,91,,GC,O'CONNOR Ben AG2R Citroën Team,27,AG2R Citroën Team,110,50,4″,0:040:044″
3,4,96,+4:02:40,106,,Sprint,PHILIPSEN Jasper Alpecin - Deceuninck,25,Alpecin - Deceuninck,90,40,,0:390:39
4,5,101,+4:12:21,86,,Classic,PEDERSEN Mads Lidl - Trek,27,Lidl - Trek,70,32,,",,0:39"
5,6,82,+3:48:16,5,,Classic,LAPORTE Christophe Jumbo-Visma,30,Jumbo-Visma,55,26,,",,0:39"
6,7,112,+4:25:03,167,,Sprint,MEZGEC Luka Team Jayco AlUla,35,Team Jayco AlUla,45,22,,",,0:39"
7,8,81,+3:48:15,43,,Classic,BETTIOL Alberto EF Education - EasyPost,29,EF Education - EasyPost,40,18,,",,0:39"
8,9,103,+4:13:05,18,,Classic,TRENTIN Matteo UAE Team Emirates,33,UAE Team Emirates,35,14,,",,0:39"
9,10,13,+44:44,26,,Classic,PIDCOCK Thomas INEOS Grenadiers,23,INEOS Grenadiers,30,10,,",,0:39"


In [116]:
def get_race_info(url):
    
    soup = get_soup(url)
    
    ul_elements = soup.find('ul', class_='infolist')
    ul_points = ul_elements.find_all('li')
    
    results = {}
    
    for point in ul_points:
        divs = point.find_all('div')
        
        key = divs[0].text.strip().replace(':','')
        val = divs[1].text
        
        results[key] = val

    return results

In [125]:
url = format_url('tour-de-france', 2021, 5)

In [164]:
def get_time_gap(race = 'tour-de-france', year = 2023, stage = None):
    
    url = format_url(race, year, stage)
    
    #soupify
    soup = get_soup(url)
    
    #find rows
    rows = soup.find_all('tr')
    
    #generate headers
    headers = rows[0]
    
    row2 = rows[2].find_all('td')
    
    for i, header in enumerate(headers): 
        if header.text == 'Time':
            time_gap = row2[i].text

    return time_gap

get_time_gap('tour-de-france', 2022, 1)

'0:050:05'